In [1]:
%pip install openai opencv-python 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from IPython.display import display, Image, Audio

from openai import OpenAI
import cv2
import base64
import time
import openai
import os
import requests

In [3]:
os.environ["OPENAI_API_KEY"] = "" # Set your API key here

In [8]:
def get_frames(video_file_path):

  video = cv2.VideoCapture(video_file_path)

  frames = []
  while video.isOpened():
      s, frame = video.read()
      if s:
        _, buffer = cv2.imencode(".jpg", frame)
        frames.append(base64.b64encode(buffer).decode("utf-8"))
      else:
         break

  video.release()

  return frames, len(frames)

In [9]:
video_file_path = "" # Set the path to your video file here

In [10]:
base64Frames, frame_count = get_frames(video_file_path=video_file_path)

In [14]:
def generate_commentary(base64Frames, user_prompt):

  client = OpenAI() # defaults to os.environ.get("OPENAI_API_KEY")

  PROMPT_MESSAGES = [
      {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": user_prompt,
            },
            
            *map(lambda x: {"type": "image_url", "image_url": {
                "url": f"data:image/jpeg;base64,{x}"    
            }}, base64Frames[0::30]),
          ],
      },
  ]
  params = {
      "model": "gpt-4-vision-preview",
      "messages": PROMPT_MESSAGES,
      "max_tokens": 200,
  }

  result = client.chat.completions.create(**params)
  print(result.choices[0].message.content)

  return result.choices[0].message.content

In [12]:
word_count = frame_count // 15

In [15]:
user_prompt = f"""
These are frames from Counter Strike match, create an exhilerating commentary on them. The commentary should be {word_count} words.
The player name is "simple".
"""

transcript = generate_commentary(base64Frames, user_prompt)

"S1mple with a stunning clutch, takes down two! Unbelievable play! Team Liquid prevails, crowd goes wild! What a magnificent end to the half!"


In [16]:
transcript

'"S1mple with a stunning clutch, takes down two! Unbelievable play! Team Liquid prevails, crowd goes wild! What a magnificent end to the half!"'

In [18]:
def transcript_to_audio(transcript):
  response = requests.post(
    "https://api.openai.com/v1/audio/speech",
    headers={
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
    },
    json={
        "model": "tts-1",
        "input": transcript,
        "voice": "alloy",
    },
  )

  audio = b""
  for chunk in response.iter_content(chunk_size=1024 * 1024):
      audio += chunk

  return audio


In [19]:
audio = transcript_to_audio(transcript)

In [20]:
with open('commentary.wav', 'wb') as file:
    # wav_data = audio.get_wav_data()
    file.write(audio)

In [21]:
Audio(audio)